
# Django - Customise Your SQL



## Josh Smeaton
<hr>
- jarshwah on #django / Twitter / Github / Slack
- Software Engineer at Engage/LivePerson
- SQL Nerd
- Django Core Developer
- Primary author of Expressions


## Recent Developments
<hr>
- Lookups (1.7)

```python
Product.objects.filter(tagarray__contains=['large', 'jumper'])
```

- Transforms (1.7)

```python
Product.objects.filter(name__lower__contains='jumper')
```

- Expressions (1.8)

```python
Sale.objects.annotate(
    sale_month=TruncMonth('sale_date')
).values('sale_month').annotate(sales_per_month=Count('id'))
```


# Sales Reporting

In [ ]:
from IPython.display import display
from shop.functions import table, sql

# Show sales and revenue per category
qs = Sale.objects.values('product__category').annotate(
    sales=Count('id'),
    revenue=Sum('sale_price')
)

display(table(qs))


## We're not interested in Kids products

In [ ]:
from IPython.display import display
from shop.functions import table, sql

# Show sales and revenue per category, but not Kids
qs = Sale.objects.exclude(
    product__category=Product.KIDS
).values('product__category').annotate(
    sales=Count('id'),
    revenue=Sum('sale_price')
)

display(table(qs))

## WEBSCALE!
<hr>

```sql
SELECT "shop_product"."category",
       SUM("shop_sale"."sale_price") AS "revenue",
       COUNT("shop_sale"."id") AS "sales"
FROM "shop_sale"
INNER JOIN "shop_product" ON ("shop_sale"."product_id" = "shop_product"."id")
WHERE NOT ("shop_product"."category" = Kids)
GROUP BY "shop_product"."category"
```

- "NOT (condition = value)" vs "condition != value" 5 characters!
- Django has to do a lot more work for (correct) negated conditions

In [ ]:
from django.db.models import Lookup
from django.db.models.fields import Field

@Field.register_lookup
class NotEqual(Lookup):
    lookup_name = 'ne'

    def as_sql(self, compiler, connection):
        lhs, lhs_params = self.process_lhs(compiler, connection)
        rhs, rhs_params = self.process_rhs(compiler, connection)
        params = lhs_params + rhs_params
        return '%s != %s' % (lhs, rhs), params
    

In [ ]:
%timeit -r 3 list(Product.objects.exclude(category='Kids'))
%timeit -r 3 list(Product.objects.filter(category__ne='Kids'))

In [ ]:
from IPython.display import display
from shop.functions import sql, dataframe
df = dataframe(Product.objects.values().order_by('cost_price'))
display(df)

In [ ]:
from shop.functions import sql, dataframe
print(sql(Product.objects.all()))